# Vehicle Detection Project

This sub project converts yolo model to keras.

## Import Packages

In [1]:
import argparse
import configparser
import io
import os
from collections import defaultdict
import colorsys
import random
import matplotlib.image as mpimg
import cv2
import numpy as np

from keras import backend as K
from keras.layers import (Conv2D, GlobalAveragePooling2D, Input, Lambda,
                          MaxPooling2D,Add,UpSampling2D,Concatenate)
from keras.layers import (Conv2D, GlobalAveragePooling2D, Input, Reshape,
                          UpSampling2D, Activation)
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.utils.vis_utils import plot_model as plot
from keras.models import load_model
from keras.layers.merge import concatenate, add


import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## Global variables

yolov2 config file can get from https://github.com/pjreddie/darknet/blob/master/cfg/yolov2.cfg

yolov2 wights file can get from https://pjreddie.com/media/files/yolov2.weights

yolov3 config file can get from https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg

yolov3 wights file can get from https://pjreddie.com/media/files/yolov3.weights

In [2]:
#yolov2
# config_path = 'yolov2.cfg'
# weights_path = 'yolov2.weights'

#yolov3
config_path = 'yolov3.cfg'
weights_path = 'yolov3.weights'
anchors_path='model_data/yolov3_anchors.txt'

classes_path='model_data/coco_classes.txt'
# anchors_path='model_data/yolo_anchors.txt'
output_root = 'model_data/yolo.h5'

## Cfg File Parse

In [3]:
def unique_config_sections(config_file):
    """Convert all config sections to have unique names.

    Adds unique suffixes to config sections for compability with configparser.
    """
    section_counters = defaultdict(int)
    output_stream = io.StringIO()
    with open(config_file) as fin:
        for line in fin:
            if line.startswith('['):
                section = line.strip().strip('[]')
                _section = section + '_' + str(section_counters[section])
                section_counters[section] += 1
                line = line.replace(section, _section)
            output_stream.write(line)
    output_stream.seek(0)
    return output_stream

In [4]:
print('Parsing Darknet config.')
unique_config_file = unique_config_sections(config_path)
cfg_parser = configparser.ConfigParser()
cfg_parser.read_file(unique_config_file)

print(cfg_parser.sections())

Parsing Darknet config.
['net_0', 'convolutional_0', 'convolutional_1', 'convolutional_2', 'convolutional_3', 'shortcut_0', 'convolutional_4', 'convolutional_5', 'convolutional_6', 'shortcut_1', 'convolutional_7', 'convolutional_8', 'shortcut_2', 'convolutional_9', 'convolutional_10', 'convolutional_11', 'shortcut_3', 'convolutional_12', 'convolutional_13', 'shortcut_4', 'convolutional_14', 'convolutional_15', 'shortcut_5', 'convolutional_16', 'convolutional_17', 'shortcut_6', 'convolutional_18', 'convolutional_19', 'shortcut_7', 'convolutional_20', 'convolutional_21', 'shortcut_8', 'convolutional_22', 'convolutional_23', 'shortcut_9', 'convolutional_24', 'convolutional_25', 'shortcut_10', 'convolutional_26', 'convolutional_27', 'convolutional_28', 'shortcut_11', 'convolutional_29', 'convolutional_30', 'shortcut_12', 'convolutional_31', 'convolutional_32', 'shortcut_13', 'convolutional_33', 'convolutional_34', 'shortcut_14', 'convolutional_35', 'convolutional_36', 'shortcut_15', 'convo

# weights

In [5]:
# Load weights and config.
print('Loading weights.')
weights_file = open(weights_path, 'rb')
weights_header = np.ndarray(
    shape=(5, ), dtype='int32', buffer=weights_file.read(20))
print('Weights Header: ', weights_header)
weights_file.close()

Loading weights.
Weights Header:  [       0        2        0 32013312        0]


# Construct the network

In [6]:
def space_to_depth_x2(x):
    """Thin wrapper for Tensorflow space_to_depth with block_size=2."""
    # Import currently required to make Lambda work.
    # See: https://github.com/fchollet/keras/issues/5088#issuecomment-273851273
    import tensorflow as tf
    return tf.space_to_depth(x, block_size=2)


def space_to_depth_x2_output_shape(input_shape):
    """Determine space_to_depth output shape for block_size=2.

    Note: For Lambda with TensorFlow backend, output shape may not be needed.
    """
    return (input_shape[0], input_shape[1] // 2, input_shape[2] // 2, 4 *
            input_shape[3]) if input_shape[1] else (input_shape[0], None, None,
                                                    4 * input_shape[3])

In [8]:
print('Creating Keras model.')
weights_file = open(weights_path, 'rb')
weights_header = np.ndarray(
    shape=(5, ), dtype='int32', buffer=weights_file.read(20))
print('Weights Header: ', weights_header)

image_height = int(cfg_parser['net_0']['height'])
image_width = int(cfg_parser['net_0']['width'])

# prev_layer = Input(shape=(image_height, image_width, 3))
# all_layers = [prev_layer]

input_layer = Input(shape=(image_height, image_width, 3))
prev_layer = input_layer
all_layers = []

weight_decay = float(cfg_parser['net_0']['decay']) if 'net_0' in cfg_parser.sections() else 5e-4

count = 0
out_index = []
    
for section in cfg_parser.sections():
    print('Parsing section {}'.format(section))
    if section.startswith('convolutional'):
#         [convolutional]
#         batch_normalize=1
#         filters=32
#         size=3
#         stride=1
#         pad=1
#         activation=leaky
        batch_normalize = 'batch_normalize' in cfg_parser[section]
        filters = int(cfg_parser[section]['filters'])
        size = int(cfg_parser[section]['size'])
        stride = int(cfg_parser[section]['stride'])
        pad = int(cfg_parser[section]['pad'])
        activation = cfg_parser[section]['activation']
        
        # padding='same' is equivalent to Darknet pad=1
        padding = 'same' if pad == 1 else 'valid'

        # Setting weights.
        # Darknet serializes convolutional weights as:
        # [bias/beta, [gamma, mean, variance], conv_weights]
        prev_layer_shape = K.int_shape(prev_layer)

        # TODO: This assumes channel last dim_ordering.
        weights_shape = (size, size, prev_layer_shape[-1], filters)
        darknet_w_shape = (filters, weights_shape[2], size, size)
        weights_size = np.product(weights_shape)

#         print('conv2d', 'bn'
#               if batch_normalize else '  ', activation, weights_shape)

        conv_bias = np.ndarray(shape=(filters, ),dtype='float32',buffer=weights_file.read(filters * 4))
        count += filters

        if batch_normalize:
            bn_weights = np.ndarray(shape=(3, filters),dtype='float32',buffer=weights_file.read(filters * 12))
            count += 3 * filters

            # TODO: Keras BatchNormalization mistakenly refers to var
            # as std.
            bn_weight_list = [
                bn_weights[0],  # scale gamma
                conv_bias,  # shift beta
                bn_weights[1],  # running mean
                bn_weights[2]  # running var
            ]

        conv_weights = np.ndarray(shape=darknet_w_shape,dtype='float32',buffer=weights_file.read(weights_size * 4))
        count += weights_size

        # DarkNet conv_weights are serialized Caffe-style:
        # (out_dim, in_dim, height, width)
        # We would like to set these to Tensorflow order:
        # (height, width, in_dim, out_dim)
        # TODO: Add check for Theano dim ordering.
        conv_weights = np.transpose(conv_weights, [2, 3, 1, 0])
        conv_weights = [conv_weights] if batch_normalize else [conv_weights, conv_bias]

        # Handle activation.
        act_fn = None
        if activation == 'leaky':
            pass  # Add advanced activation later.
        elif activation != 'linear':
            raise ValueError(
                'Unknown activation function `{}` in section {}'.format(
                    activation, section))

        # Create Conv2D layer
        conv_layer = (Conv2D(filters, (size, size),strides=(stride, stride),kernel_regularizer=l2(weight_decay),\
            use_bias=not batch_normalize,weights=conv_weights,activation=act_fn,padding=padding))(prev_layer)

        if batch_normalize:
            conv_layer = (BatchNormalization(weights=bn_weight_list))(conv_layer)
        prev_layer = conv_layer

        if activation == 'linear':
            all_layers.append(prev_layer)
        elif activation == 'leaky':
            act_layer = LeakyReLU(alpha=0.1)(prev_layer)
            prev_layer = act_layer
            all_layers.append(act_layer)

    elif section.startswith('maxpool'):
#         [maxpool]
#         size=2
#         stride=2
        size = int(cfg_parser[section]['size'])
        stride = int(cfg_parser[section]['stride'])
        all_layers.append(MaxPooling2D(padding='same',pool_size=(size, size),strides=(stride, stride))(prev_layer))
        prev_layer = all_layers[-1]

    elif section.startswith('avgpool'):
        if cfg_parser.items(section) != []:
            raise ValueError('{} with params unsupported.'.format(section))
        all_layers.append(GlobalAveragePooling2D()(prev_layer))
        prev_layer = all_layers[-1]

    elif section.startswith('route'):
#         [route]
#         layers=-9
        ids = [int(i) for i in cfg_parser[section]['layers'].split(',')]
        layers = [all_layers[i] for i in ids]
        if len(layers) > 1:
            print('Concatenating route layers:', layers)
            concatenate_layer = concatenate(layers)
            all_layers.append(concatenate_layer)
            prev_layer = concatenate_layer
        else:
            skip_layer = layers[0]  # only one layer to route
            all_layers.append(skip_layer)
            prev_layer = skip_layer

    elif section.startswith('reorg'):
        block_size = int(cfg_parser[section]['stride'])
        assert block_size == 2, 'Only reorg with stride 2 supported.'
        all_layers.append(
            Lambda(space_to_depth_x2,output_shape=space_to_depth_x2_output_shape,name='space_to_depth_x2')(prev_layer))
        prev_layer = all_layers[-1]

    elif section.startswith('region'):
        with open('{}_anchors.txt'.format(output_root), 'w') as f:
            print(cfg_parser[section]['anchors'], file=f)
        
    elif section.startswith('shortcut'):
        index = int(cfg_parser[section]['from'])
        activation = cfg_parser[section]['activation']
        assert activation == 'linear', 'Only linear activation supported.'
        all_layers.append(Add()([all_layers[index], prev_layer]))
        prev_layer = all_layers[-1]
        
    elif section.startswith('upsample'):
        stride = int(cfg_parser[section]['stride'])
        assert stride == 2, 'Only stride=2 supported.'
        all_layers.append(UpSampling2D(stride)(prev_layer))
        prev_layer = all_layers[-1]

    # TODO: Further implement needed.
    elif section.startswith('yolo'):
        out_index.append(len(all_layers)-1)
        all_layers.append(None)
        prev_layer = all_layers[-1]

    elif (section.startswith('net') or section.startswith('cost') or
          section.startswith('softmax')):
        pass  # Configs not currently handled during model definition.

    else:
        raise ValueError(
            'Unsupported section header type: {}'.format(section))

# Create and save model.
# model = Model(inputs=all_layers[0], outputs=all_layers[-1])
model = Model(inputs=input_layer, outputs=[all_layers[i] for i in out_index])
model.save('{}'.format(output_root))
print('Saved Keras model to {}'.format(output_root))
# Check to see if all weights have been read.
remaining_weights = len(weights_file.read()) / 4
weights_file.close()
print('Read {} of {} from Darknet weights.'.format(count, count + remaining_weights))

print("Construction Done!")

Creating Keras model.
Weights Header:  [       0        2        0 32013312        0]
Parsing section net_0
Parsing section convolutional_0
Parsing section convolutional_1
Parsing section convolutional_2
Parsing section convolutional_3
Parsing section shortcut_0
Parsing section convolutional_4
Parsing section convolutional_5
Parsing section convolutional_6
Parsing section shortcut_1
Parsing section convolutional_7
Parsing section convolutional_8
Parsing section shortcut_2
Parsing section convolutional_9
Parsing section convolutional_10
Parsing section convolutional_11
Parsing section shortcut_3
Parsing section convolutional_12
Parsing section convolutional_13
Parsing section shortcut_4
Parsing section convolutional_14
Parsing section convolutional_15
Parsing section shortcut_5
Parsing section convolutional_16
Parsing section convolutional_17
Parsing section shortcut_6
Parsing section convolutional_18
Parsing section convolutional_19
Parsing section shortcut_7
Parsing section convolutio

In [9]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv2d_76 (Conv2D)              (None, 416, 416, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_73 (BatchNo (None, 416, 416, 32) 128         conv2d_76[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_73 (LeakyReLU)      (None, 416, 416, 32) 0           batch_normalization_73[0][0]     
__________________________________________________________________________________________________
conv2d_77 